# **Очистка  и обработка данных**

In [51]:
import pandas as pd

Загрузим датасет, который мы получили после парсинга данных со стены сообщества.

In [52]:
df = pd.read_csv('project_dataset.csv')
df.head()

,Unnamed: 0,date,likes,reposts,views,comments,attachments,text
0,31701,1681698600,786,401,112018,18,9,🖤 Изабель Аджани в фильме «Смертельная поездка...
1,51009,1714739400,46,30,5841,3,9,"«Чернокнижник: Армагеддон», 1993 год. \nРежисс..."
2,50998,1714725000,274,561,31437,3,9,"«Губная помада», 1976 год. \nРежиссер: Ламонт ..."
3,50989,1714714200,114,51,13382,1,9,В главной роли — Роми Шнайдер 📽
4,50983,1714703400,154,23,13978,1,6,🖤 Маргарет Куолли в объективе Тома Шумахера 🖤


Опишем полученные данные в таблице по столбцам:

*   `unnamed: 0` - id поста
*   `date` - дата поста
*   `likes` - количество лайков
*   `reposts` - количество репостов
*   `views` - количество просмотров
*   `comments` - количество комментариев
*   `attachments` - количество вложений
*   `text` - текст поста


Дальше преобразуем столбец text в несколько разных. Посты в паблике делятся на два типа: те, в которых есть информация о фильме и режиссере, а также те, в которых есть просто фотографии актеров. Первый тип можно дополнительно обработать, вычленив название фильма, имя режиссера и год производства фильма благодаря достаточно строгому naming convention у авторов паблика. С помощью регулярных выражений выделим отдельные столбцы с названием фильмов, именами режиссеров и годами производства.
Иногда после этой информации еще есть дополнительное описание сюжета фильма, который мы вынесли в столбец additional info также для постов, в которых нет строгой информации о фильмах и режиссерах.

In [53]:
df['movie'] = df['text'].str.extract(r'^«(.+?)»')
df['director'] = df['text'].str.extract(r'Режисс[ёе]р(?:ы)?: (.+?)(?:\.|$)')
df['director'] = df['director'].str.replace(r',', '|').str.split('|').apply(lambda x: ', '.join([i.strip() for i in x]) if isinstance(x, list) else x)
df['year'] = df['text'].str.extract(r'(\d{4})')

In [54]:
df['additional_info'] = df['text'].str.replace(r'^«(.+?)»|Режисс[ёе]р(?:ы)?: (.+?)\.|\d{4}|год', '', regex=True).str.strip()
df['additional_info'] = df['additional_info'].str.replace(r'^\s*,\s*', '', regex=True)
df['additional_info'] = df['additional_info'].str.replace(r'\n', '', regex=True)
df['additional_info'] = df['additional_info'].str.replace(r'^\.', '', regex=True)

На всякий случай преобразуем формат столбца с датами.

In [55]:
df['year'] = pd.to_numeric(df['year'], errors='coerce').fillna(0).astype(int)
df.head()

,Unnamed: 0,date,likes,reposts,views,comments,attachments,text,movie,director,year,additional_info
0,31701,1681698600,786,401,112018,18,9,🖤 Изабель Аджани в фильме «Смертельная поездка...,NaN,NaN,1983,🖤 Изабель Аджани в фильме «Смертельная поездка...
1,51009,1714739400,46,30,5841,3,9,"«Чернокнижник: Армагеддон», 1993 год. \nРежисс...",Чернокнижник: Армагеддон,Энтони Хикокс,1993,
2,50998,1714725000,274,561,31437,3,9,"«Губная помада», 1976 год. \nРежиссер: Ламонт ...",Губная помада,Ламонт Джонсон,1976,"Марго Хемингуэй играет модель Крис, которая у..."
3,50989,1714714200,114,51,13382,1,9,В главной роли — Роми Шнайдер 📽,NaN,NaN,0,В главной роли — Роми Шнайдер 📽
4,50983,1714703400,154,23,13978,1,6,🖤 Маргарет Куолли в объективе Тома Шумахера 🖤,NaN,NaN,0,🖤 Маргарет Куолли в объективе Тома Шумахера 🖤


Мы также собрали датасет по трем самым престижным премиям: Oскару, Золотой пальмовой ветви и Золотому глобусу. По Oскару были доступны данные на русском, в том числе по номинантам, а по Золотой пальмовой ветви и Золотому глобусу - только по лауреатам, так что мы внесем данные по номинантам и лауреатам Оскара, лауреатам Золотой пальмовой ветви и Золотого глобуса.

Начнем с данных по Оскару.

In [56]:
oscar = pd.read_excel('oscar.xlsx')
oscar.head()

,Год/Церемония,Фильм,Кинокомпания(и),Продюсер(ы),Победитель
0,1929,«Крылья»,Paramount Pictures,Люсьен Хаббард,1
1,1929,«Рэкет»,Paramount Pictures,Говард Хьюз,0
2,1929,«Седьмое небо»,Fox Film Corporation,Уильям Фокс,0
3,1930,«Бродвейская мелодия»,Metro-Goldwyn-Mayer,"Ирвинг Талберг, Лоуренс Уэйнгартен",1
4,1930,«Алиби»,United Artists,Роланд Уэст,0


Тут формат данных по названию фильма не совпадал с тем, что есть у нас в таблице, поэтому преобразуем названия в таблице oscar к нашему формату.

In [57]:
oscar['Фильм'] = oscar['Фильм'].str.replace('«', '').str.replace('»', '').str.strip()

Все фильмы в датасете - номинанты. Поэтому сделаем еще один столбец-флаг того, что фильм-номинант.

In [58]:
oscar['oscar_nominee'] = 1

Тут мы убираем ненужные столбцы, которые будут затормаживать процесс склейки таблиц.

In [59]:
oscar = oscar.drop(['Кинокомпания(и)', 'Продюсер(ы)', 'Год/Церемония'], axis=1)

Джойн таблиц слева, потому что мы не хотим потерять данные по фильмам, которые не были номинированы.

In [60]:
df1 = pd.merge(df, oscar, left_on='movie', right_on='Фильм', how='left')
df1 = df1.rename(columns = {'Победитель' : 'oscar_winner'})
df1.head()

,Unnamed: 0,date,likes,reposts,views,comments,attachments,text,movie,director,year,additional_info,Фильм,oscar_winner,oscar_nominee
0,31701,1681698600,786,401,112018,18,9,🖤 Изабель Аджани в фильме «Смертельная поездка...,NaN,NaN,1983,🖤 Изабель Аджани в фильме «Смертельная поездка...,NaN,NaN,NaN
1,51009,1714739400,46,30,5841,3,9,"«Чернокнижник: Армагеддон», 1993 год. \nРежисс...",Чернокнижник: Армагеддон,Энтони Хикокс,1993,,NaN,NaN,NaN
2,50998,1714725000,274,561,31437,3,9,"«Губная помада», 1976 год. \nРежиссер: Ламонт ...",Губная помада,Ламонт Джонсон,1976,"Марго Хемингуэй играет модель Крис, которая у...",NaN,NaN,NaN
3,50989,1714714200,114,51,13382,1,9,В главной роли — Роми Шнайдер 📽,NaN,NaN,0,В главной роли — Роми Шнайдер 📽,NaN,NaN,NaN
4,50983,1714703400,154,23,13978,1,6,🖤 Маргарет Куолли в объективе Тома Шумахера 🖤,NaN,NaN,0,🖤 Маргарет Куолли в объективе Тома Шумахера 🖤,NaN,NaN,NaN


Мы заметили, что часто строки, содержащие название фильма, который был номинирован, все равно маркируются как отсутствующие данные в столбцах про номинантов и победителей премии. Поэтому мы написали функцию, которая это будет поправлять.

In [61]:
def update_oscar_nominee(row):
    if pd.notnull(row['movie']) and row['oscar_nominee'] != 1:
        return 0
    else:
        return row['oscar_nominee']

df1['oscar_nominee'] = df1.apply(update_oscar_nominee, axis=1)

То же самое касалось и столбца победителей премии.

In [62]:
def update_oscar_winner(row):
    if row['oscar_nominee'] == 0:
        return 0
    elif pd.isna(row['oscar_winner']):
        return 0
    else:
        return row['oscar_winner']

df1['oscar_winner'] = df1.apply(update_oscar_winner, axis = 1)

Теперь проанализируем лауреатов премии **Золотая пальмовая ветвь**.

In [63]:
palme = pd.read_excel('palme.xlsx')
palme.head()

,Год,Название фильма,Режиссер
0,1939,Юнион Пасифик,Сесил Би Демилль
1,1946,Травля,Альф Шёберг
2,1946,Потерянный уикенд,Билли Уайлдер
3,1946,Красные луга,"Бодиль Ипсен, Лау Лауритцен мл."
4,1946,Город в долине,Четан Ананд


Снова добавим флаг того, что фильм в строке - лауреат.

In [64]:
palme['palme_winner'] = 1

Склеим с уже имеющимся датасетом.

In [65]:
df2 = pd.merge(df1, palme, left_on='movie', right_on='Название фильма', how='left')

Отбросим ненужные столбцы, которые дублируют информацию.

In [66]:
df2 = df2.drop(['Год', 'Название фильма', 'Режиссер'], axis = 1)

Здесь вновь была проблема с тем, что не все фильмы маркировались верно, поэтому мы опять применили функцию для более корректной маркировки победителей и фильмов, не получивших награду.

In [67]:
def update_palme_winner(row):
    if pd.notnull(row['movie']) and row['palme_winner'] != 1:
        return 0
    else:
        return row['palme_winner']

df2['palme_winner'] = df2.apply(update_palme_winner, axis=1)

Переименовываем столбец для более корректного отображения информации о датасете.

In [68]:
df2 = df2.rename(columns = {'Unnamed: 0' : 'post_id'})

Теперь добавим информацию о лауреатах премии **Золотой глобус**.

In [69]:
golden_globe = pd.read_excel('golden_globe.xlsx')
golden_globe.head()

,Год,Фильм,Режиссёр,Продюсер,Победитель
0,1944,Песня Бернадетт,Генри Кинг,Уильям Перлберг,1
1,1945,Идти своим путём,Лео Маккэри,Лео Маккэри,1
2,1946,Потерянный уикэнд,Билли Уайлдер,Чарльз Брэкетт,1
3,1947,Лучшие годы нашей жизни,Уильям Уайлер,Сэмюэл Голдвин,1
4,1948,Джентльменское соглашение,Элиа Казан,Дэррил Занук,1


Джойним два датасета по названию фильма и переименовываем столбец для соблюдения naming convention.

In [70]:
df3 = pd.merge(df2, golden_globe, left_on='movie', right_on='Фильм', how='left')
df3 = df3.rename(columns = {'Победитель' : 'globe_winner'})

golden_globe.columns

Index(['Год', 'Фильм', 'Режиссёр', 'Продюсер', 'Победитель'], dtype='object')

Отбрасываем ненужные столбцы, которые содержат лишнюю информацию или дублируют ее.

In [71]:
df3 = df3.drop(['Год', 'Фильм_y','Продюсер', 'Режиссёр', 'Фильм_x'], axis = 1)

Поскольку в дальнейшем нам пригодится знание о том, какие строки содержат фильмы, то мы добавим столбец с флагом наличия фильма.

In [72]:
df3['movie_flg'] = pd.notnull(df3['movie'])

Нам вновь потребовалось избавиться от большого количества отсутствующих данных посредством заполнения столбца победителей с помощью функции.

In [73]:
def update_globe_winner(row):
    if pd.notnull(row['movie']) and row['globe_winner'] == 1:
        return 1
    elif pd.notnull(row['movie']) and row['globe_winner'] != 1:
        return 0
    else:
      return row['globe_winner']

df3['globe_winner'] = df3.apply(update_globe_winner, axis=1)

Сохраним обработанные данные.

In [74]:
df3.to_csv('processed_dataset.csv', index=False)